In [ ]:
from numpy import sqrt, array, min, ones, shape, pi # Numpy elements for calculations
from numpy.random import rand # For the seed value for the diagonalization
from scipy.sparse import csr_matrix #import sparse matrix type csr
from scipy.sparse.linalg import lobpcg # Diagonalization method
# from scipy.linalg import norm # For calculating the norm of a vector
from sympy.physics.quantum.cg import CG # For calculating Clebsch-Gordan coefficients
from sympy import simplify # To obtain simplified expresions with the Clebsc-Gordan
from iminuit import Minuit # For the minimization routine

import glob
import matplotlib.pyplot as plt
import time
%matplotlib inline

import time

start_time = time.time()

In [ ]:
# We define a function that generates the input file for ArbModel for the needed matrices
def inputArbmodel(params, h = 'H'):

    boson_number = params[0]
    f = open("216Ra_op.in","w")
    pre = """
    # some general settings
    FolderOfISF             ./ISFDataBase
    # the model:
    ParticleTypeNumber       4
    ParticleTypeNames        s  p  d  f
    ParticleTypeParities     1 -1  1 -1
    ParticleTypeSymmetries   1  3  5  7
    ParticleTypeMaxNums     -1 -1 -1 -1
    ParticleTypeSeniorities -1 -1 -1 -1
    # the states to calculate:
    TotalParticleNumber     {}
    TotalSpin               0  2  4  6  8 10 12 14 16 18 20 22 24 
    TotalParity             1 -1  1 -1  1 -1  1 -1  1 -1  1 -1  1
    NumberOfStatesToCalc   -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
    UseHamiltonian         H1 H1 H1 H1 H1 H1 H1 H1 H1 H1 H1 H1 H1
    # tell the code which diagonalisation it should use
    DiagMethod              1  1  1  1  1  1  1  1  1  1  1  1  1
    # tell the code what to print out
    PrintBasis              n  n  n  n  n  n  n  n  n  n  n  n  n  
    PrintEigenvalues        y  y  y  y  y  y  y  y  y  y  y  y  y 
    PrintEigenvectors       n  n  n  n  n  n  n  n  n  n  n  n  n  
    PrintInputsummary       n
    VerboseLevel            0
    Groundstate             1
    # the hamiltonian
    """.format(boson_number)
    f.write(pre)
    ed,ep,ef,k2,k1,k3,chi,chisq = params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8]
    tra = params[9]
    f = open("216Ra_op.in","a")
    f.write("WriteHamiltToFile h0.txt h1.txt h2.txt h3.txt h4.txt h5.txt h6.txt h7.txt h8.txt h9.txt h10.txt h11.txt h12.txt \n".replace('h',h))
    f.write("WriteOperatorsToFile op_0.txt op_1.txt op_2.txt op_3.txt op_4.txt op_5.txt op_6.txt op_7.txt op_8.txt op_9.txt op_10.txt op_11.txt op_12.txt \n".replace("op", tra))
    f.write( nd(ed))
    f.write( np(ep))
    f.write( nf(ef))
    f.write(QQ2(k2))
    f.write(QQ2dd(chi))
    f.write(QQ2dd2(chisq))
    f.write(QQ1(k1))
    f.write(QQ3(k3))
    f.write(op(tra))
    f.close()

# The following functions are the definition of the different operators used
    
def nd(coef):
    n =  "H H1 d + d ~ %15.15e" % (sqrt(5.0)*coef) + "\n"
    return n

def np(coef):
    n =  "H H1 p + p ~ %15.15e" % (sqrt(3.0)*coef) + "\n"
    return n
    
def nf(coef):
    n =  "H H1 f + f ~ %15.15e" % (sqrt(7.0)*coef) + "\n"
    return n

def QQ2(coef):
    s, p, d, f = "s","p","d","f"
    q2 = [
        [s,d,1],
        [d,s,1],
        [d,d, 0],
        [p,f, -((3.0*sqrt(7.0))  / 5  )],
        [f,p, -((3.0*sqrt(7.0))  / 5  )],
        [p,p,-((9.0*sqrt(3.0))  / 10 )],
        [f,f,-((3.0*sqrt(42.0)) / 10 )]
        ] 
    qq = ""
    for first in q2:
        for second in q2:
            component = "H H1 " + first[0] + ' + ' + first[1] + ' ~ ' + second[0] + ' + ' + second[1] + ' ~ 4 %15.15e' % (coef*first[2]*second[2]*sqrt(5.0)) + "\n"
            qq += component
    return qq

def QQ2dd(coef):
    s, p, d, f = "s","p","d","f"

    qdd = [
        [s,d,0],
        [d,s,0],
        [d,d,1],
        [p,f,0],
        [f,p,0],
        [p,p,0],
        [f,f,0]
        ] 


    q2 = [
        [s,d,1],
        [d,s,1],
        [d,d,0],
        [p,f, -((3.0*sqrt(7.0))  / 5  )],
        [f,p, -((3.0*sqrt(7.0))  / 5  )],
        [p,p,-((9.0*sqrt(3.0))  / 10 )],
        [f,f,-((3.0*sqrt(42.0)) / 10 )]
        ] 
    qq = ""
    for first in qdd:
        for second in q2:
            component = "H H1 " + first[0] + ' + ' + first[1] + ' ~ ' + second[0] + ' + ' + second[1] + ' ~ 4 %15.15e' % (coef*first[2]*second[2]*sqrt(5.0)) + "\n"
            qq += component
    qq2 = ""
    for first in q2:
        for second in qdd:
            component = "H H1 " + first[0] + ' + ' + first[1] + ' ~ ' + second[0] + ' + ' + second[1] + ' ~ 4 %15.15e' % (coef*first[2]*second[2]*sqrt(5.0)) + "\n"
            qq2 += component
    return qq + qq2

def QQ2dd2(coef):
    return "H H1 d + d ~ d + d ~ 4 %15.15e" % (coef*sqrt(5.0)) + "\n"
    
def QQ1(coef):
    s, p, d, f = "s","p","d","f"
    q = [
        [s,p, sqrt(10.0) ],
        [p,s,-sqrt(10.0) ],
        [d,p, sqrt(8.0)  ],
        [p,d,-sqrt(8.0)  ],
        [d,f, sqrt(42.0) ],
        [f,d,-sqrt(42.0) ]
        ]
    qq = ""
    for first in q:
        for second in q:
            component = "H H1 " + first[0] + ' + ' + first[1] + ' ~ ' + second[0] + ' + ' + second[1] + ' ~ 2 %15.15e' % (coef*first[2]*second[2]*(-sqrt(3.0))) + "\n"
            qq += component
    return qq

def QQ3(coef):
    s, p, d, f = "s","p","d","f"
    q = [
        [s,f, -sqrt(5)],
        [f,s,  sqrt(5)],
        [d,p,  -2     ],
        [p,d,   2     ],
        [d,f, -sqrt(6)],
        [f,d,  sqrt(6)]
        ]
    qq = ""
    for first in q:
        for second in q:
            component = "H H1 " + first[0] + ' + ' + first[1] + ' ~ ' + second[0] + ' + ' + second[1] + ' ~ 6 %15.15e' % (coef*first[2]*second[2]*(-sqrt(7.0))) + "\n"
            qq += component
    return qq

def op(coef):
    if coef == "E3":
        op = "O E3 p + d ~ 6 2.0 d + p ~ 6 -2.0 f + s ~ 6 2.23606797749979 s + f ~ 6 -2.23606797749979 f + d ~ 6 2.449489742783178 d + f ~ 6 -2.449489742783178 \n M  1 1 E3 4 1 \n M  3 1 E3 6 1 \n M  5 1 E3 8 1 \n M 7 1 E3 10 1 \n M 2 1 E3 3 1 \n M 3 1 E3 4 1 \n M 4 1 E3 5 1 \n"
    elif coef == "E1":
        op = "O E1 d + p ~ 2 2.82842712474619 p + d ~ 2 -2.82842712474619 s + p ~ 2 3.162277660168379 p + s ~ 2 -3.162277660168379 d + f ~ 2 6.48074069840786 f + d ~ 2 -6.48074069840786 \n M 1 1 E1 2 1 \n M 2 1 E1 3 1 \n M 3 1 E1 4 1 \n M 4 1 E1 5 1 \n M 5 1 E1 6 1 \n M 6 1 E1 7 1 \n M 7 1 E1 8 1 \n"
    elif coef == "E2":
        op = "O E2 s + d ~ 4 1.0 d + s ~ 4 1.0 d + d ~ 4 0.0 p + f ~ 4 -1.587450786638754 f + p ~ 4 -1.587450786638754 p + p ~ 4 -1.55884572681199 f + f ~ 4 -1.944222209522358 \n M 3 1 E2 1 1 \n M 5 1 E2 3 1 \n M 7 1 E2 5 1 \n M 9 1 E2 7 1 \n M 11 1 E2 9 1 \n M  4 1 E2 2 1 \n M  6 1 E2 4 1 \n"
    elif coef == "E2dd":
        op = "O E2 s + d ~ 4 0.0 d + s ~ 4 0.0 d + d ~ 4 1.0 p + f ~ 4 0.0 f + p ~ 4 0.0 p + p ~ 4 0.0 f + f ~ 4 0.0 \n M 3 1 E2 1 1 \n M 5 1 E2 3 1 \n M 7 1 E2 5 1 \n M 9 1 E2 7 1 \n M 11 1 E2 9 1 \n M  4 1 E2 2 1 \n M  6 1 E2 4 1 \n"
    else:
        op = '#'
    return op

# For loading the matrices in sparse format:
    
def matrix(operator):
    yale_matrix = []
    f = open(operator)
    for line in f:        
        yale_matrix.append(line.split())
    f.close()
    Data=array(yale_matrix[5])
    Data=Data.astype('float64')
    IA=array(yale_matrix[3])
    IA=IA.astype('float64')
    JA=array(yale_matrix[4])
    JA=JA.astype('float64')
    M=csr_matrix((Data, JA, IA))
    return M


def run_arbmodel():
    %run run_arbmodel.ipynb
    return True

def generator(N):
    # # To generate a file, one uses the following order [N,nd,np,nf,Q2,Q1,Q3,Q2_dd,Q2_dd2,EX], including a "one" in the desired space for the desired matrix. In the case of the EX, one should include "E1", "E2", "E2dd", "E3" depending on the desired transition.
    params = [N,1,0,0,0,0,0,0,0,'0']
    inputArbmodel(params, h = 'ED_')
    run_arbmodel()
    print("New ED matrices")
    params = [N,0,1,0,0,0,0,0,0,'0']
    inputArbmodel(params, h = 'EP_')
    run_arbmodel()
    print("New EP matrices")
    params = [N,0,0,1,0,0,0,0,0,'0']
    inputArbmodel(params, h = 'EF_')
    run_arbmodel()
    print("New EF matrices")
    params = [N,0,0,0,1,0,0,0,0,'0']
    inputArbmodel(params, h = 'Q2_')
    run_arbmodel()
    print("New Q2 matrices")
    params = [N,0,0,0,0,1,0,0,0,'0']
    inputArbmodel(params, h = 'Q1_')
    run_arbmodel()
    print("New Q1 matrices")
    params = [N,0,0,0,0,0,1,0,0,'0']
    inputArbmodel(params, h = 'Q3_')
    run_arbmodel()
    print("New Q3 matrices")
    params = [N,0,0,0,0,0,0,1,0,'0']
    inputArbmodel(params, h = 'Q2dd_')
    run_arbmodel()
    print("New Q2dd matrices")
    params = [N,0,0,0,0,0,0,0,1,'0']
    inputArbmodel(params, h = 'Q2dd2_')
    run_arbmodel()
    print("New Q2dd2 matrices")
    params = [N,0,0,0,0,0,0,0,0,"E3"]
    inputArbmodel(params)
    run_arbmodel()
    print("New E3 matrices")
    params = [N,0,0,0,0,0,0,0,0,"E1"]
    inputArbmodel(params)
    run_arbmodel()
    print("New E1 matrices")
    params = [N,0,0,0,0,0,0,0,0,"E2"]
    inputArbmodel(params)
    run_arbmodel()
    print("New E2 matrices")
    params = [N,0,0,0,0,0,0,0,0,"E2dd"]
    inputArbmodel(params)
    run_arbmodel()
    print("New E2dd matrices")
    print('Matrix update!')
    return True

def matrix_load():

    # To load the matrices obtained with ArbModel in memory
        
    ed_op,ep_op,ef_op,q2_op,q1_op,q3_op,q2dd_op,q2dd2_op = [],[],[],[],[],[],[],[]
    e3_op, e2_op, e2dd_op, e1_op = [],[],[],[]
    
    for L in range(0,13):
        ed_op.append(matrix(glob.glob("N8_matrix/chi/ED_{}.txt".format(L))[0]))
        ep_op.append(matrix(glob.glob("N8_matrix/chi/EP_{}.txt".format(L))[0]))
        ef_op.append(matrix(glob.glob("N8_matrix/chi/EF_{}.txt".format(L))[0]))
        q2_op.append(matrix(glob.glob("N8_matrix/chi/Q2_{}.txt".format(L))[0]))
        q1_op.append(matrix(glob.glob("N8_matrix/chi/Q1_{}.txt".format(L))[0]))
        q3_op.append(matrix(glob.glob("N8_matrix/chi/Q3_{}.txt".format(L))[0]))
        q2dd_op.append(matrix(glob.glob("N8_matrix/chi/Q2dd_{}.txt".format(L))[0]))
        q2dd2_op.append(matrix(glob.glob("N8_matrix/chi/Q2dd2_{}.txt".format(L))[0]))


    for j in range(0,7):
        e3_op.append(matrix(glob.glob("N8_matrix/chi/E3_{}.txt".format(j))[0]))
        e2_op.append(matrix(glob.glob("N8_matrix/chi/E2_{}.txt".format(j))[0]))
        e2dd_op.append(matrix(glob.glob("N8_matrix/chi/E2dd_{}.txt".format(j))[0]))
        e1_op.append(matrix(glob.glob("N8_matrix/chi/E1_{}.txt".format(j))[0]))

    return ed_op,ep_op,ef_op,q2_op,q1_op,q3_op,q2dd_op,q2dd2_op,e3_op,e2_op,e2dd_op,e1_op

# Functions defined to calculate the B(EL) in Weisskopf units

def wu_estimate(l, A):
    return (((1.2)**(2*l))/(4*pi))*((3/(l+3))**2) * (A**(2*l/3))

def wu(matrix, l, A, Li):
    B = matrix**2/(2*Li+1)
    return B/wu_estimate(l,A)

# To calculate the energies, once the minimization is finished

def Energies(ed, ep, ef, k2, k1, k3, chi, e1, e2, e3):
    E = []
    eigenvectors = []
    for L in range(13):
        H = ed*ed_op[L] + ep*ep_op[L] + ef*ef_op[L] - k2*(q2_op[L] + chi*q2dd_op[L] + (chi**2)*q2dd2_op[L]) - k1*q1_op[L]  - k3*q3_op[L]

        # seed(42)
        X= rand(H.shape[0], 15)
        w, v = lobpcg(H, X=X, largest=False, tol=1e-3)
        v0 = v[:,0]
        if (L==0):
            E_min = min(w.real) 
        E.append(min(w.real)-E_min)
        eigenvectors.append(v0)

    # For the matrix elements and the electromagnetic transitions, in efm^L.

    TE1 = []

    BE1 = []

    TE1_10 = abs(eigenvectors[0].T @ (e1*e1_op[0]) @ eigenvectors[1])

    TE1.append(TE1_10)

    BE1.append(TE1_10**2/3)

    TE1_12 = abs(eigenvectors[1].T @ (e1*e1_op[1]) @ eigenvectors[2])

    TE1.append(TE1_12)

    BE1.append(TE1_12**2/3)

    TE1_32 = abs(eigenvectors[2].T @ (e1*e1_op[2]) @ eigenvectors[3])

    TE1.append(TE1_32)

    BE1.append(TE1_32**2/7)

    TE1_34 = abs(eigenvectors[3].T @ (e1*e1_op[3]) @ eigenvectors[4])

    TE1.append(TE1_34)

    BE1.append(TE1_34**2/7)

    TE1_54 = abs(eigenvectors[4].T @ (e1*e1_op[4]) @ eigenvectors[5])

    TE1.append(TE1_54)

    BE1.append(TE1_54**2/11)

    TE1_65 = abs(eigenvectors[5].T @ (e1*e1_op[5]) @ eigenvectors[6])

    TE1.append(TE1_65)

    BE1.append(TE1_65**2/13)

    TE2 = []

    BE2 = []

    TE2_20 = abs(eigenvectors[2].T @ (e2*(e2_op[0] + chi*e2dd_op[0])) @ eigenvectors[0])

    TE2.append(TE2_20)

    BE2.append(TE2_20**2/5)

    TE2_42 = abs(eigenvectors[4].T @ (e2*(e2_op[1] + chi*e2dd_op[1])) @ eigenvectors[2])

    TE2.append(TE2_42)

    BE2.append(TE2_42**2/9)

    TE2_64 = abs(eigenvectors[6].T @ (e2*(e2_op[2] + chi*e2dd_op[2])) @ eigenvectors[4])

    TE2.append(TE2_64)

    BE2.append(TE2_64**2/13)

    TE2_86 = abs(eigenvectors[8].T @ (e2*(e2_op[3] + chi*e2dd_op[3])) @ eigenvectors[6])

    TE2.append(TE2_86)

    BE2.append(TE2_86**2/17)

    TE2_31 = abs(eigenvectors[3].T @ (e2*(e2_op[5] + chi*e2dd_op[5])) @ eigenvectors[1])

    TE2.append(TE2_31)

    BE2.append(TE2_31**2/7)

    TE2_53 = abs(eigenvectors[5].T @ (e2*(e2_op[6] + chi*e2dd_op[6])) @ eigenvectors[3])

    TE2.append(TE2_53)

    BE2.append(TE2_53**2/11)

    TE3 = []

    BE3 = []

    TE3_30 = abs(eigenvectors[0].T @ (e3*e3_op[0]) @ eigenvectors[3])

    TE3.append(TE3_30)

    BE3.append(TE3_30**2/7)

    TE3_52 = abs(eigenvectors[2].T @ (e3*e3_op[1]) @ eigenvectors[5])

    TE3.append(TE3_52)

    BE3.append(TE3_52**2/11)

    TE3_74 = abs(eigenvectors[4].T @ (e3*e3_op[2]) @ eigenvectors[7])

    TE3.append(TE3_74)

    BE3.append(TE3_74**2/15)

    TE3_12 = abs(eigenvectors[1].T @ (e3*e3_op[4]) @ eigenvectors[2])

    TE3.append(TE3_12)

    BE3.append(TE3_12**2/3)

    TE3_32 = abs(eigenvectors[2].T @ (e3*e3_op[5]) @ eigenvectors[3])

    TE3.append(TE3_32)

    BE3.append(TE3_32**2/7)

    # We divide by the wu_estimate function to obtain the transition rates in Weisskopf units

    return E, TE1, TE2, TE3, array(BE1)/wu_estimate(1,216), array(BE2)/wu_estimate(2,216), array(BE3)/wu_estimate(3,216)        

# To calculate the multipole intrinsic moments

def multipole(order, Ii, If, i):
    cg = CG(Ii, 0, order, 0, If, 0)
    const1 = sqrt(3/(4*pi))
    const23 = sqrt((2*order+1)/(16*pi))
    if order == 1:
        Q = abs(sqrt(BE1[i]*wu_estimate(1,216))/(cg.doit()*const1))
    elif order == 2:
        Q = abs(sqrt(BE2[i]*wu_estimate(2,216))/(cg.doit()*const23))
    elif order == 3:
        Q = abs(sqrt(BE3[i]*wu_estimate(3,216))/(cg.doit()*const23))
    else:
        Q = 0
    return simplify(Q.evalf())



In [ ]:
# We define the desired cost function

def cost_fuction(ed,ep,ef,k2,k1,k3,chi, e1, e2, e3):
    minimization_time = time.time()

    E = 0

    # Experimental energies 

    # 216Ra
    # exp = [0, float("NaN"), 0.688, float("NaN"), 1.164, float("NaN"), 1.507, float("NaN"), 1.711,
    #        float("NaN"), 2.026, 2.335, float("NaN")]

    # 218Ra
    # exp = [0, 0.853, 0.388, 0.793, 0.741, 1.038, 1.122, 1.340, 1.546,
    #        1.694, 1.961, 2.109, 2.390]

    # E_err = [1, 0.006, 0.0001, 0.00018, 0.00014, 0.000018, 0.000020, 0.000021, 0.000023, 0.000025, 0.00003, 0.00003, 0.00003]

    # 220Ra
    # exp = [0, 0.412, 0.178, 0.474, 0.410, 0.634, 0.688, 0.873, 1.001,
    #        1.163, 1.342, 1.496, 1.711]
    
    # E_err = [1, 0.00010, 0.00012, 0.00023, 0.00023, 0.0004, 0.0003, 0.0004, 0.00004, 0.00004, 0.00005, 0.00005, 0.00005]

    # 222Ra
    # exp = [0, 0.242, 0.111, 0.317, 0.301, 0.473, 0.550, 0.703, 0.843,
    #        0.914, 1.173, 1.330, 1.537]

    # E_err = [1, 0.000017, 0.00002, 0.000022, 0.000034, 0.00004, 0.00019, 0.00027, 0.00025, 0.00033, 0.00003, 0.00004, 0.00005]

    
    # 224Ra
    exp = [0, 0.215, 0.084, 0.290, 0.250, 0.433, 0.479, 0.640, 0.754,
           0.906, 1.068, 1.220, 1.413]

    E_err = [1, 0.000017, 0.00002, 0.000022, 0.000034, 0.00004, 0.00019, 0.00027, 0.00025, 0.00033, 0.00003, 0.00004, 0.00005]

    
    # 226Ra
    # exp = [0, 0.253, 0.067, 0.321, 0.211, 0.446, 0.416, 0.626, 0.669,
    #        0.857, 0.959, 1.133, 1.280]
    
    # E_err = [1, 0.00001, 0.00001, 0.00006, 0.00002, 0.0002, 0.0003, 0.0002, 0.0003, 0.0006, 0.0003, 0.00003, 0.00004]

    # 228Ra
    # exp = [0, 0.474, 0.064, 0.538, 0.205, 0.656, 0.412, 0.830, 0.674,
    #        1.055, 0.983, 1.327, 1.331]

    # E_err = [1, 0.00004, 0.0002, 0.00005, 0.000022, 0.00005, 0.00005, 0.0005, 0.00011, 0.00005, 0.00015, 0.00004, 0.00004]

    L216 = [0,2,4,6,8,10,11]

    energies = []
    eigenvectors = []

    # In this case, we calculate the states between L=0 and L=13. It can be modified upon desire

    for L in range(0,13):
    
    # To work with 216Ra, given the data available, one should comment the 'range' line and instead uncomment and use the following line 'L216'

    # for L in L216:
        H = ed*ed_op[L] + ep*ep_op[L] + ef*ef_op[L] - k2*(q2_op[L] + chi*q2dd_op[L] + chi**2*q2dd2_op[L]) - k1*q1_op[L] - k3*q3_op[L]
        X= rand(H.shape[0], 5)
        w, v = lobpcg(H, X=X, largest=False, tol=1e-3)
        if (L==0):
            E_min = min(w.real) 
        E += ( (min(w.real) - E_min)- exp[L] )**2/(E_err[L]**2) # Definition of the cost function for the energies, weighed by their respective errors.
        v0 = v[:,0]
        w0 = w[0].real - E_min
        energies.append(w0)
        eigenvectors.append(v0)

    # e3_op = [0: 3->0, 1: 5->2, 2: 7->4, 3: 9->6, 4: 2->1, 5: 3->2, 6: 4->3]

    # e2_op = [0: 2->0, 1: 4->2, 2: 6->4, 3: 8->6, 4: 10->8, 5: 3->1, 6: 5->3]

    # e1_op = [0: 1->0, 1: 2->1, 2: 3->2, 3: 4->3, 4: 5->4, 5: 6->5, 6: 7->6]

    # Definition of the matrix elements. We use an absolute value here.

    TE1_10 = abs(eigenvectors[0].T @ (e1*e1_op[0]) @ eigenvectors[1])

    TE1_12 = abs(eigenvectors[1].T @ (e1*e1_op[1]) @ eigenvectors[2])

    TE1_32 = abs(eigenvectors[2].T @ (e1*e1_op[2]) @ eigenvectors[3])

    TE1_34 = abs(eigenvectors[3].T @ (e1*e1_op[3]) @ eigenvectors[4])

    TE1_54 = abs(eigenvectors[4].T @ (e1*e1_op[4]) @ eigenvectors[5])

    TE1_65 = abs(eigenvectors[5].T @ (e1*e1_op[5]) @ eigenvectors[6])

    TE3_30 = abs(eigenvectors[0].T @ (e3*e3_op[0]) @ eigenvectors[3])

    TE3_52 = abs(eigenvectors[2].T @ (e3*e3_op[1]) @ eigenvectors[5])

    TE3_74 = abs(eigenvectors[4].T @ (e3*e3_op[2]) @ eigenvectors[7])

    TE3_12 = abs(eigenvectors[1].T @ (e3*e3_op[4]) @ eigenvectors[2])

    TE3_32 = abs(eigenvectors[2].T @ (e3*e3_op[5]) @ eigenvectors[3])

    TE2_20 = abs(eigenvectors[2].T @ (e2*(e2_op[0] + chi*e2dd_op[0])) @ eigenvectors[0])

    TE2_42 = abs(eigenvectors[4].T @ (e2*(e2_op[1] + chi*e2dd_op[1])) @ eigenvectors[2])

    TE2_64 = abs(eigenvectors[6].T @ (e2*(e2_op[2] + chi*e2dd_op[2])) @ eigenvectors[4])

    TE2_86 = abs(eigenvectors[8].T @ (e2*(e2_op[3] + chi*e2dd_op[3])) @ eigenvectors[6])

    TE2_31 = abs(eigenvectors[3].T @ (e2*(e2_op[5] + chi*e2dd_op[5])) @ eigenvectors[1])

    TE2_53 = abs(eigenvectors[5].T @ (e2*(e2_op[6] + chi*e2dd_op[6])) @ eigenvectors[3])

    # Basis. For each nuclei, only include the transitions that have experimental data available.

    # BE1_theo = [TE1_10, TE1_12, TE1_32, TE1_34, TE1_54, TE1_65]

    # BE2_theo = [TE2_20, TE2_42, TE2_64, TE2_86, TE2_31, TE2_53]

    # BE3_theo = [TE3_30, TE3_52, TE3_74, TE3_12, TE3_32]

    # 216Ra does not present experimental values that may be reliable
            
    # 218Ra

    # BE1_theo = [TE1_65]

    # BE1_exp = [0.42]

    # BE1_error = [0.06]

    # BE2_theo = [TE2_20, TE2_42, TE2_64]

    # BE2_exp = [99.69, 210.22, 215.89]

    # BE2_error = [4.69, 20.02, 25.81]
        
    # 220Ra does not present experimental values available.

    # 222Ra

    # BE1_theo = [TE1_10, TE1_12, TE1_32, TE1_34, TE1_54, TE1_65]

    # BE1_exp = [0.082, 0.114, 0.26, 0.27, 0.180, 0.277]

    # BE1_error = [0.007, 0.02, 0.05, 0.26, 0.0335, 0.011]

    # BE2_theo = [TE2_20, TE2_42, TE2_64, TE2_86, TE2_31, TE2_53]

    # BE2_exp = [211, 298, 357, 415, 235, 310]

    # BE2_error = [9, 15, 18, 23, 22, 40]

    # BE3_theo = [TE3_30, TE3_52, TE3_74, TE3_12, TE3_32]

    # BE3_exp = [1130, 1790, 3300, 850, 900]

    # BE3_error = [90, 200, 400, 240, 500]

    # 224Ra

    BE1_theo = [TE1_10, TE1_32, TE1_54]

    BE1_exp = [0.018, 0.026, 0.030]

    BE1_error = [0.018, 0.005, 0.001]

    BE2_theo = [TE2_20, TE2_42, 
                TE2_64, TE2_86, 
                TE2_31, TE2_53]

    BE2_exp = [199, 230,
                315, 410,
                405, 500]

    BE2_error = [3, 11,
                  6, 60,
                  15, 60]

    BE3_theo = [TE3_30, TE3_52, TE3_12]

    BE3_exp = [940, 1410, 1370]

    BE3_error = [30, 190, 140]

    # 226Ra

    # BE1_theo = [TE1_10, TE1_12, TE1_32, TE1_34, TE1_54]

    # BE1_exp = [0.050, 0.068, 0.061, 0.057, 0.093]

    # BE1_error = [0.009, 0.01, 0.007, 0.005, 0.007]

    # BE2_theo = [TE2_20, TE2_42, 
    #             TE2_64, TE2_86, 
    #             TE2_31, TE2_53]

    # BE2_exp = [226, 370,
    #            483, 541,
    #             366, 409]

    # BE2_error = [1, 7,
    #               6, 9,
    #               9, 7]

    # BE3_theo = [TE3_30, TE3_52,
    #              TE3_74,
    #                TE3_12, TE3_32]

    # BE3_exp = [1080, 2000,
    #             2450,
    #               1190, 1150]

    # BE3_error = [30, 70,
    #               105,
    #                 50, 160]

    ## 228Ra

    # BE1_theo = [TE1_10, TE1_12, TE1_32, TE1_34, TE1_54, TE1_65]

    # BE1_exp = [0.043, 0.051, 0.09, 0.08, 0.13, 0.074]

    # BE1_error = [0.026, 0.0275, 0.04, 0.0295, 0.06, 0.015]

    # BE2_theo = [TE2_20**2/(5*wu_estimate(2,228)), TE2_42, TE2_64, TE2_86, TE2_31, TE2_53]

    # BE2_exp = [142, 387, 511, 589, 380, 390]

    # BE2_error = [6, 19, 26, 29, 50, 60]

    # BE3_theo = [TE3_30, TE3_52, TE3_12
    #             # , TE3_32
    #             ]

    # BE3_exp = [870, 1710, 1360
    #         #    , 60
    #            ]

    # BE3_error = [150, 230, 230
    #             #  , 195
    #              ]

                  
    # Cost functions for the electromagnetic transitions, weighed by their respective experimental error.

    B1 = 0
    B2 = 0
    B3 = 0

    for i in range(0,len(BE1_exp)):
        B1 += (BE1_theo[i]- BE1_exp[i])**2/BE1_error[i]**2
    
    for i in range(0,len(BE2_exp)):
        B2 += (BE2_theo[i]- BE2_exp[i])**2/BE2_error[i]**2
    
    for i in range(0,len(BE3_exp)):
        B3 += (BE3_theo[i]- BE3_exp[i])**2/BE3_error[i]**2
    
    # Generate a temporal file with the values in the current iteration of the minimization routine.

    f = open("temporal_results_224.in","w")
    f.write("224Ra \n")
    f.write('ed = ' + str(ed) + '\n')
    f.write('ep = ' + str(ep) + '\n')
    f.write('ef = ' + str(ef) + '\n')
    f.write('k2 = ' + str(k2) + '\n')
    f.write('k1 = ' + str(k1) + '\n')
    f.write('k3 = ' + str(k3) + '\n')
    f.write('chi = ' + str(chi) + '\n')
    f.write('Energies = ' + str(energies) + '\n')
    f.write('TE1 (efm) = ' + str(BE1_theo) + '\n')
    f.write('TE2 (efm2) = ' + str(BE2_theo) + '\n')
    f.write('TE3 (efm3) = ' + str(BE3_theo) + '\n')
    f.write('e1 = ' + str(e1) + '\n')
    f.write('e2 = ' + str(e2) + '\n')
    f.write('e3 = ' + str(e3) + '\n')
    f.write('Residuals: \n')
    f.write('E = ' + str(E) + '\n')
    f.write('B1 = ' + str(B1) + '\n')
    f.write('B2 = ' + str(B2) + '\n')
    f.write('B3 = ' + str(B3) + '\n')
    f.close()

    total_time = time.time() - minimization_time

    print("One iteration completed. It took " + str(total_time) + ' seconds')
    print("The program has been running for " + str((time.time() - start_minuit)) + " seconds")

    # Return the cost function. The numerical weights included are to priorize one transition over another. These values are the ones used for most of the isotopes on the chain.

    return E*0.01 + B2*100 + B3*1000 + B1*10

In [ ]:
# Generates the desired matrices. The number is changed in function of the nucleus to be used.

generator(8)

In [ ]:
# Load matrices into memory

ed_op,ep_op,ef_op,q2_op,q1_op,q3_op,q2dd_op,q2dd2_op,e3_op,e2_op,e2dd_op,e1_op = matrix_load()

In [ ]:
start_minuit = time.time()

# Set the desired cost function and the seed values for the minimization

m = Minuit(cost_fuction, 0.263, 0.8, 0.460, 0.03, 0.00, 0.0068, -1.231, 0.0039, 14.3, 300)

# Set the limits for each parameter

m.limits = [(0.16,0.6), (0.7,1.5), (0.4,0.8), (0.025,0.035), (0.00,0.01), (0.00,0.01), (-3,-1.2), (0,0.1), (10, 50), (50, 300)]



# One may fix a given parameter to the seed value, if needed.

# m.fixed['ed'] = True
# m.fixed['ep'] = True
# m.fixed['ef'] = True
# m.fixed['k2'] = True
# m.fixed['k1'] = True
# m.fixed['k3'] = True
# m.fixed['chi'] = True
m.fixed['e1'] = True
m.fixed['e2'] = True
m.fixed['e3'] = True

# The minimization itself

m.migrad()

In [ ]:
# Print the parameters (notebook only)

m.params

In [ ]:
ep = m.values['ep']
ed = m.values['ed']
ef = m.values['ef']
k1 = m.values['k1']
k2 = m.values['k2']
k3 = m.values['k3']
chi = m.values['chi']
e1 = m.values['e1']
e2 = m.values['e2']
e3 = m.values['e3']


E, TE1, TE2, TE3, BE1, BE2, BE3 = Energies(ed, ep, ef, k2, k1, k3, chi, e1, e2, e3)

Q1 = []
Q2 = []
Q3 = []

Q1.append(multipole(1,1,0,0))
Q1.append(multipole(1,1,2,1))
Q1.append(multipole(1,3,2,2))
Q1.append(multipole(1,3,4,3))
Q1.append(multipole(1,5,4,4))
Q1.append(multipole(1,6,5,5))

Q2.append(multipole(2,2,0,0))
Q2.append(multipole(2,4,2,1))
Q2.append(multipole(2,6,4,2))
Q2.append(multipole(2,8,6,3))
Q2.append(multipole(2,3,1,4))
Q2.append(multipole(2,5,3,5))

Q3.append(multipole(3,3,0,0))
Q3.append(multipole(3,5,2,1))
Q3.append(multipole(3,7,4,2))
Q3.append(multipole(3,1,2,3))
Q3.append(multipole(3,3,2,4))

# We print into terminal our obtained results

print('Energies (MeV) = ', E)
print('TE1 (efm) = ', TE1)
print('TE2 (efm2) = ', TE2)
print('TE3 (efm3) = ', TE3)
print('BE1 (Wu) = ', BE1)
print('BE2 (Wu) = ', BE2)
print('BE3 (Wu) = ', BE3)
print('ed (MeV) = ', ed)
print('ep (MeV) = ', ep)
print('ef (MeV) = ', ef)
print('k2 (MeV) = ', k2)
print('k1 (MeV) = ', k1)
print('k3 (MeV) = ', k3)
print('e1 (efm) = ', e1)
print('e2 (efm2) = ', e2)
print('e3 (efm3) = ', e3)
print('chi = ', chi)
print('Q1 (efm) = ', Q1)
print('Q2 (efm2) = ', Q2)
print('Q3 (efm3) = ', Q3)

In [ ]:
# Generate an output file with all the desired results

output = open("218_chi2B3_sparse_full_output_thesisv2.out","w")
output.write('Energies (MeV) = ' + str(E) + '\n')
output.write('T(E1) (efm)  = ' + str(TE1) + '\n')
output.write('T(E2) (efm2) = ' + str(TE2) + '\n')
output.write('T(E3) (efm3) = ' + str(TE3) + '\n')
output.write('B(E1) (W.u.) = ' + str(BE1) + '\n')
output.write('B(E2) (W.u.) = ' + str(BE2) + '\n')
output.write('B(E3) (W.u.) = ' + str(BE3) + '\n')
output.write('Q1 (efm) = ' + str(Q1) + '\n')
output.write('Q2 (efm2) = ' + str(Q2) + '\n')
output.write('Q3 (efm3) = ' + str(Q3) + '\n')
output.write('ed = ' + str(ed) + '\n')
output.write('ep = ' + str(ep) + '\n')
output.write('ef = ' + str(ef) + '\n')
output.write('k2 = ' + str(k2) + '\n')
output.write('k1 = ' + str(k1) + '\n')
output.write('k3 = ' + str(k3) + '\n')
output.write('chi = ' + str(chi) + '\n')
output.write('e1 = ' + str(e1) + '\n')
output.write('e2 = ' + str(e2) + '\n')
output.write('e3 = ' + str(e3) + '\n')
# output.write('Minimization time = ' + str(migrad_runtime) + ' seconds \n')
output.close()

In [ ]:
# To plot the energy spectra for a better comprehension of the obtained results


    # 216Ra
# exp = [0, float("NaN"), 0.688, float("NaN"), 1.164, float("NaN"), 1.507, float("NaN"), 1.711,
#        float("NaN"), 2.026, 2.335, float("NaN")]

    # 218Ra
# exp = [0, 0.853, 0.388, 0.793, 0.741, 1.038, 1.122, 1.340, 1.546,
#            1.694, 1.961, 2.109, 2.390]

    # 220Ra
# exp = [0, 0.412, 0.178, 0.474, 0.410, 0.634, 0.688, 0.873, 1.001,
#            1.163, 1.342, 1.496, 1.711]

    # 222Ra
# exp = [0, 0.242, 0.111, 0.317, 0.301, 0.473, 0.550, 0.703, 0.843,
#            0.914, 1.173, 1.330, 1.537]
    
    # 224Ra
exp = [0, 0.215, 0.084, 0.290, 0.250, 0.433, 0.479, 0.640, 0.754,
           0.906, 1.068, 1.220, 1.413]
    
    # 226Ra
# exp = [0, 0.253, 0.067, 0.321, 0.211, 0.446, 0.416, 0.626, 0.669,
        #    0.857, 0.959, 1.133, 1.280]

    #228Ra
# exp = [0, 0.474, 0.064, 0.538, 0.205, 0.656, 0.412, 0.830, 0.674,
#            1.055, 0.983, 1.327, 1.331]

# E = [0.0, 
#  0.23932518489975596, 
#  0.10678293460378852, 
#  0.30922516848404946, 
#  0.3028813169123099, 
#  0.5087528211571937, 
#  0.5259425275375871, 
#  0.7181451437449766, 
#  0.7782358175074928, 
#  1.038779275989489, 
#  1.1191366759205403, 
#  1.3061518611071707, 
#  1.5236526268541635]

exp_odd = []
exp_even = []
theo_odd = []
theo_even = []
L_odd = []
L_even = []

for i in range(0,len(E)):
    if i % 2 == 0:
        exp_even.append(exp[i])
        theo_even.append(E[i])
        L_even.append(str(i)+'$^+$')
    else:
        exp_odd.append(exp[i])
        theo_odd.append(E[i])
        L_odd.append(str(i)+'$^⁻$')


x_odd = ones(len(exp_odd))*1
x_even = ones(len(exp_even))*0

y_odd = ones(len(theo_odd))*3
y_even = ones(len(theo_even))*2

fig, ax = plt.subplots(1, 1)

plt.rcParams['figure.figsize'] = [10, 6]
ax.scatter(x_odd, exp_odd, marker = '_', s=4000, linewidth=2, color='red')
ax.set_xlim(-0.5, 3.5)
# ax.set_ylim(0.0, 2.1)

ax.scatter(y_odd, theo_odd, marker = '_', s=4000, linewidth=2, color='red')
ax.scatter(x_even, exp_even, marker = '_', s=4000, linewidth=2, color='blue')
ax.scatter(y_even, theo_even, marker = '_', s=4000, linewidth=2, color='blue')

# plt.scatter(z, zero)
ax.set_xticks([0.5, 2.5])
ax.set_xticklabels(["Exp", "Theo"])
ax.set_ylabel('Energy [MeV]')

# Crea las etiquetas L^{signo}


    
for d, e, f in zip(y_odd, theo_odd, L_odd):
    plt.annotate(f, xy=(3.3, e))

# L_odd = ['11$^-$']

for a, b, c in zip(x_odd, exp_odd, L_odd):
    plt.annotate(c, xy=(1.3, b))

for g, h, i in zip(x_even, exp_even, L_even):
    plt.annotate(i, xy=(0.3, h))

# L_even.append('12$^+$')

for j, k, l in zip(y_even, theo_even, L_even):
    plt.annotate(l, xy=(2.3, k))
   
ax.set_title('$^{224}$Ra')
    
plt.savefig("224Ra_chi_sparse_0303.pdf")